### REFERENCES:
1. For understanding purpose: https://www.geeksforgeeks.org/viterbi-algorithm-for-hidden-markov-models-hmms/

### THEORY:
1. Used to find the most probable sequence of hidden states in a Hidden Markov Model (HMM).
2. Used in speech recognition, natural language processing and bioinformatics.
3. Hidden Markov Models (HMMs):
   1. It is a statistical model that represents systems with hidden states and observable events. It consists of:
      1. States: Hidden conditions that are not directly observable.
      2. Observations: Observable events infulenced by the hidden states.
      3. Transition Probabilities: Probabilities of moving from one state to another.
      4. Emission Probabilities: Probabilities of observing a particular event given a state.
      5. Initial State Probabilties: Probabilities of the system starting in a particular state.
4. Andrew Viterbi, who first proposed the algorithm in 1967, is honored by the algorithm's name.
5. The most likely sequence of states is computed efficiently by the Viterbi method, which divided the issue into smaller subproblems and solves them recursively.
6. The Viterbi algorithm operates by iteratively computing the highest probability paths to each state at each time step, storing these probabilities, and backtracking to determine the most probable sequence of hidden states.

**Now let's work on the code**:

In [37]:
# Defining HMM Parameters:
# 1. States -> E (exon) and I (intron)
import numpy as np
states = ['E', '5', 'I']
nucleotides = ['A', 'C', 'G', 'T']
# 2. Initial Probabilities: Probability of starting with an E or an I
# Considering equal probabilities for both the states in the starting
initial_probabilities = {'E': 1.0, '5': 0.0, 'I': 0.0}
# 3. Transition matrix: Probabilities of moving from one state to another
transition_probabilities = {
    'Start': {'E':1.0, '5': 0, 'I': 0.0, 'End':0.0},
    'E': {'E': 0.9, '5': 0.1 , 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0 , 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0 , 'I': 0.9, 'End': 0.1}
}
# 4. Emission probabilities: Probability of emitting nucleotides (A, C, G, T) in each state
emission_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.00, 'G': 0.95, 'T': 0.00},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},
}

In [41]:
import math
def log(x):
    if(x == 0):
        return -math.inf
    else:
        return math.log(x)

In [42]:
# Log probability of given path: This function will be used to compute the log probability of a state path emitting an observed sequence. (base 2)
def get_log_prob_of_a_given_path(state_path, observed_sequence):
    log_prob = 0.0
    if len(state_path)!=len(observed_sequence):
        raise ValueError("The length of state path and the observed sequence must be same")
    prev_state = 'Start'
    # log_prob += log(initial_probabilities[prev_state])
    # log_prob += log(emission_probs[prev_state][observed_sequence[0]])

    for i in range(len(observed_sequence)):
        current_state = state_path[i]
        observed_state = observed_sequence[i]
        log_prob += log(transition_probabilities[prev_state][current_state])+log(emission_probs[current_state][observed_state])
        prev_state = current_state
    if(prev_state == 'I'):
        log_prob += log(transition_probabilities[prev_state]['End'])

    return log_prob

In [43]:
# state_path = "EEEEEEEEEEEEEEEEEEE5IIIIIII"
state_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
ans = get_log_prob_of_a_given_path(state_path, observed_sequence)

print(ans)

-41.21967768602254


In [51]:
# Viterbi Algorithm Implementation
def viterbiAlgo(observed_sequence):
    num_states = len(states)
    num_observations = len(observed_sequence)
    viterbi_matrix = np.full((num_states, num_observations), -np.inf)
    # backpointers will used for backtracking (retracing the path of maxima)
    backpointers = np.zeros((num_states, num_observations), dtype=int)
    # Convert to state indices
    state_to_index = {s: i for i, s in enumerate(states)}

    # Initializing the DP table
    for i, j in enumerate(states):
        viterbi_matrix[i, 0] = log(initial_probabilities[j]) + \
        log(emission_probs[j][observed_sequence[0]])
        # Recursive part
    for k in range(1, num_observations):
        for curr_index, current_state in enumerate(states):
            max_log_prob = -np.inf
            best_prev_idx = 0
            for prev_idx, prev_state in enumerate(states):
                log_prob = viterbi_matrix[prev_idx, k-1] + \
                log(transition_probabilities[prev_state][current_state])
                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    best_prev_idx = prev_idx
            viterbi_matrix[curr_index, k] = max_log_prob + \
            log(emission_probs[current_state][observed_sequence[k]])
            backpointers[curr_index, k] = best_prev_idx

    bestPath = []
    bestFinalIndex = np.argmax(viterbi_matrix[:, -1])
    bestPath.append(states[bestFinalIndex])

    for i in range(num_observations-1, 0, -1):
        bestFinalIndex = backpointers[bestFinalIndex, i]
        bestPath.insert(0, states[bestFinalIndex])
    return bestPath, np.max(viterbi_matrix[:, -1])

In [52]:
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
best_path, log_prob = viterbiAlgo(observed_sequence)
print(f"Most probable path is {best_path} with probability {log_prob}" )

Most probable path is ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E'] with probability -38.677666280562796
